In [ ]:
import glob
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'  # 也可以使用 tensorflow
# os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,exception_verbosity=high'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cuda,optimizer=fast_compile'
os.environ["CUDA_VISIBLE_DEVICES"] = "8"

import keras.backend as K

K.set_image_data_format('channels_last')

import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, LSTM, Concatenate, Dense, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten, UpSampling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.datasets import mnist
from urllib.request import urlretrieve
from keras.optimizers import RMSprop, SGD, Adam
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.models import model_from_json

# from read_data import *
a = %pwd
print(a)

%run /root/jupyter/inspace/sang-min/myo_proejct/load_data.py import DataLoader
#%load load_data import DataLoader
#from load_data import DataLoader


'''
Model structure

Input : (100) Vector
Output : (64, 64, 1) image

Generator
100 -> 256
256 -> 16 x 16
16 x 16 -> 32 x 32
32 x 32 -> 64 x 64
64 x 64 -> 128 x 128

Discriminator
128 x 128 -> 64 x 64
64 x 64 -> 32 x 32
32 x 32 -> 16 x 16
16 x 16 -> 8 x 8
8 x 8 -> 64
64 -> 32 (16, 10)

'''
loader = DataLoader(data_path='./dataset_2018_05_16/', is_real_image=False)

conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)


def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)

    return model


def generative_model(noise_size, image_channel):
    # lstm_layer = LSTM(80, input_shape=lstm_size)(lstm_input)
    # _ = Concatenate(axis=-1)([lstm_layer, noise_input])
    print(" _ : ", noise_input)
    _ = Dense(2*2*512, input_shape=(100,), activation='relu', bias_initializer='glorot_normal',
              kernel_initializer='glorot_normal')(noise_input)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Reshape((2, 2, 512), input_shape=(2*2*512,))(_)

    _ = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=64, kernel_size=3, padding='same',  kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    #_ = Dropout(0.3)(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=32, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=16, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    #_ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=8, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    _ = Dropout(0.3)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=image_channel, kernel_size=3, padding='same', kernel_initializer=conv_init)(_)
    _ = Activation(activation='tanh')(_)

    return Model(inputs=noise_input, outputs=_)


def discriminative_model(image_size, image_channel):
    _ = inputs = Input(shape=(image_size, image_size, image_channel))

    _ = Conv2D(filters=32, kernel_size=(5, 5), border_mode='same', input_shape=(image_size,image_size, image_channel),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=64,kernel_size=(5, 5), border_mode='same', input_shape=(64, 64, 32),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(5, 5), border_mode='same', input_shape=(32, 32, 64),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=256, kernel_size=(5, 5), border_mode='same', input_shape=(16, 16, 128),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=512, kernel_size=(5, 5), border_mode='same', input_shape=(8, 8, 256),
               kernel_initializer=conv_init)(_)
    _ = AveragePooling2D(pool_size=(2, 2), border_mode='valid')(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=1, kernel_size=(2, 2), strides=1, padding='same', input_shape=(4, 4, 512),
               kernel_initializer=conv_init)(_)
    # _ = LeakyReLU(alpha=0.2)(_)

    outputs = Flatten()(_)
    outputs = Dense(1, activation='sigmoid')(outputs)

    return Model(inputs=inputs, outputs=outputs)


lstm_size = (300, 16)
noise_size = 100
image_size = 128
input_size = 100
image_channel = 1
learning_rate = 2e-4
optimizer = Adam(0.0002, 0.5)

# lstm_input = Input(shape=lstm_size)
noise_input = Input(shape=(noise_size,))
real_image = Input(shape=(image_size, image_size, image_channel))

net_g = generative_model(noise_size, image_channel)
net_g.summary()

net_d = discriminative_model(image_size, image_channel)
net_d.summary()

net_d.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

fake_image = net_g(noise_input)

net_d.trainable = False

valid = net_d(fake_image)

combined_model = Model(inputs=noise_input, outputs=valid)
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer)

# net_g.compile(loss='binary_crossentropy', optimizer='SGD')


'''
loader = DataLoader(data_path='./MYO_Dataset_label/')

emg = loader.load_emg_data()
image, label = loader.load_image()

print(emg.shape)
print(image.shape, label)

emg = loader.get_emg_datas(10)
images, labels = loader.get_images(10)
print(emg.shape, images.shape, labels.shape)

'''

epoch = 10000
i = 0
time_0 = time.time()
batch_size = 32

d_loss_history = []
g_loss_history = []
d_acc_history = []


while i <= epoch:
    # x_train = loader.get_emg_datas(batch_size)
    images = loader.get_images(batch_size)
    noise = np.random.normal(loc=0.0, scale=1.0,size=[batch_size, noise_size])

    gan_image = net_g.predict(noise)
    print("gan imaga1 : ", gan_image.shape)

    d_loss_real = net_d.train_on_batch(images, np.ones(shape=(batch_size, 1)))
    d_loss_fake = net_d.train_on_batch(gan_image, np.zeros(shape=(batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # print("epoch[%d] d_loss[%f]" % (i, d_loss))

    g_loss = combined_model.train_on_batch(noise, np.ones(shape=(batch_size, 1)))

    # print("epoch[%d] g_loss[%f]" % (i, g_loss))

    print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (i, d_loss[0], 100 * d_loss[1], g_loss))
    
    g_loss_history.append(g_loss)
    d_loss_history.append(d_loss[0])
    d_acc_history.append(100 * d_loss[1])


    if i % 500 == 0:
        gan_image = net_g.predict(noise)
        print("gan imaga2 : ", gan_image[0].shape)
        cv2.imwrite('./output_image3/' + 'fake_image'+ str(i) + '.png', gan_image[0] * 127.5)
        #cv2.imwrite('./output_image3/' + 'real_image'+ str(i) + '.png', images[0] * 127.5)
        
    i +=1;

net_g.save_weights("./output_image3/g_model.h5")
net_d.save_weights("./output_image3/d_model.h5")
combined_model.save_weights("./output_image3/combined_model.h5")

g_model_json = net_g.to_json()
with open("./output_image3/g_model.json", "w") as json_file:
    json_file.write(g_model_json)
    
d_model_json = net_d.to_json()
with open("./output_image3/d_model.json", "w") as json_file:
    json_file.write(d_model_json)

combined_model_json = combined_model.to_json()
with open("./output_image3/combined_model.json", "w") as json_file:
    json_file.write(combined_model_json)

print("Saved model to disk")

#print("save model ")
plt.figure(1, figsize=(16, 8))
plt.plot(d_loss_history)
plt.ylabel('d_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image3/d_loss_history.png')

#plt.show()

plt.figure(2, figsize=(16, 8))
plt.plot(g_loss_history)
plt.ylabel('g_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image3/g_loss_history.png')

#plt.show()\

plt.figure(3, figsize=(16, 8))
plt.plot(d_acc_history)
plt.ylabel('d_acc')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image3/d_acc_history.png')

plt.show()

print("Finish")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


/root/jupyter/inspace/sang-min/myo_proejct
30
./dataset_2018_05_16/1/
15
 _ :  Tensor("input_1:0", shape=(?, 100), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              206848    
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 2, 512)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 256)         1179904   
_________________________________________________________________
batch_normalization_2 (Batch (None, 2, 2, 256)         8         
____________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:128: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer=<keras.ini..., filters=32, padding="same", input_shape=(128, 128,..., kernel_size=(5, 5))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:129: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D(pool_size=(2, 2), padding="valid")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:134: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer=<keras.ini..., filters=64, padding="same", input_shape=(64, 64, 3..., kernel_size=(5, 5))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:135: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D(pool_size=(2, 2), padding="valid")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:140: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(ker

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 128, 32)      832       
_________________________________________________________________
average_pooling2d_1 (Average (None, 64, 64, 32)        0         
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 64, 64, 32)        256       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 64)        51264     
_________________________________________________________________
average_pooling2d_2 (Average (None, 32, 32, 64)        0         
__________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

4276 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.189751]
gan imaga1 :  (32, 128, 128, 1)
4277 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.192486]
gan imaga1 :  (32, 128, 128, 1)
4278 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.190292]
gan imaga1 :  (32, 128, 128, 1)
4279 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.192632]
gan imaga1 :  (32, 128, 128, 1)
4280 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.194318]
gan imaga1 :  (32, 128, 128, 1)
4281 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.194601]
gan imaga1 :  (32, 128, 128, 1)
4282 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.196692]
gan imaga1 :  (32, 128, 128, 1)
4283 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.196110]
gan imaga1 :  (32, 128, 128, 1)
4284 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.197433]
gan imaga1 :  (32, 128, 128, 1)
4285 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.199360]
gan imaga1 :  (32, 128, 128, 1)
4286 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.200245]
gan imaga1 :  (32, 128, 128, 1)

4366 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.294500]
gan imaga1 :  (32, 128, 128, 1)
4367 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.295790]
gan imaga1 :  (32, 128, 128, 1)
4368 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.296657]
gan imaga1 :  (32, 128, 128, 1)
4369 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.298380]
gan imaga1 :  (32, 128, 128, 1)
4370 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.299448]
gan imaga1 :  (32, 128, 128, 1)
4371 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.300726]
gan imaga1 :  (32, 128, 128, 1)
4372 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.301787]
gan imaga1 :  (32, 128, 128, 1)
4373 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.302958]
gan imaga1 :  (32, 128, 128, 1)
4374 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.303932]
gan imaga1 :  (32, 128, 128, 1)
4375 [D loss: 0.000003, acc.: 100.00%] [G loss: 12.305113]
gan imaga1 :  (32, 128, 128, 1)
4376 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.306334]
gan imaga1 :  (32, 128, 128, 1)

4456 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.401875]
gan imaga1 :  (32, 128, 128, 1)
4457 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.403823]
gan imaga1 :  (32, 128, 128, 1)
4458 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.406126]
gan imaga1 :  (32, 128, 128, 1)
4459 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.405075]
gan imaga1 :  (32, 128, 128, 1)
4460 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.409269]
gan imaga1 :  (32, 128, 128, 1)
4461 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.411822]
gan imaga1 :  (32, 128, 128, 1)
4462 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.409251]
gan imaga1 :  (32, 128, 128, 1)
4463 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.411142]
gan imaga1 :  (32, 128, 128, 1)
4464 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.411401]
gan imaga1 :  (32, 128, 128, 1)
4465 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.411560]
gan imaga1 :  (32, 128, 128, 1)
4466 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.413736]
gan imaga1 :  (32, 128, 128, 1)

4546 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.500502]
gan imaga1 :  (32, 128, 128, 1)
4547 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.502026]
gan imaga1 :  (32, 128, 128, 1)
4548 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.502888]
gan imaga1 :  (32, 128, 128, 1)
4549 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.505917]
gan imaga1 :  (32, 128, 128, 1)
4550 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.505168]
gan imaga1 :  (32, 128, 128, 1)
4551 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.507956]
./dataset_2018_05_16/13/
gan imaga1 :  (32, 128, 128, 1)
4552 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.507729]
gan imaga1 :  (32, 128, 128, 1)
4553 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.508968]
gan imaga1 :  (32, 128, 128, 1)
4554 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.510279]
gan imaga1 :  (32, 128, 128, 1)
4555 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.511722]
gan imaga1 :  (32, 128, 128, 1)
4556 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.512632]
gan im

4636 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.607582]
./dataset_2018_05_16/1/
gan imaga1 :  (32, 128, 128, 1)
4637 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.608589]
gan imaga1 :  (32, 128, 128, 1)
4638 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.609589]
gan imaga1 :  (32, 128, 128, 1)
4639 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.610525]
gan imaga1 :  (32, 128, 128, 1)
4640 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.612654]
gan imaga1 :  (32, 128, 128, 1)
4641 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.613983]
gan imaga1 :  (32, 128, 128, 1)
4642 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.614254]
gan imaga1 :  (32, 128, 128, 1)
4643 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.616454]
gan imaga1 :  (32, 128, 128, 1)
4644 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.617106]
gan imaga1 :  (32, 128, 128, 1)
4645 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.617801]
gan imaga1 :  (32, 128, 128, 1)
4646 [D loss: 0.000002, acc.: 100.00%] [G loss: 12.619194]
gan ima